In [ ]:
import nltk
nltk.download('all')
from nltk.corpus import stopwords  #stopwords
from nltk.stem import WordNetLemmatizer  
from sklearn.feature_extraction.text import TfidfVectorizer
stop_words=set(nltk.corpus.stopwords.words('english'))
import pandas as pd
import csv

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

In [ ]:
df = pd.read_csv('/content/Questions.csv', encoding='latin-1', engine='python', error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 3793: unexpected end of data


In [ ]:
df =df.drop(['OwnerUserId' ,  'CreationDate' , 'ClosedDate' , 'Score'] , axis =1)

In [ ]:
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=set(nltk.corpus.stopwords.words('english'))
df['Title']=df['Title'].apply(lambda x : x.lower())
df['Title'] = df['Title'].apply(lambda x : ' '.join([w for w in x.split() if not w in stop_words]))

In [ ]:
doc_clean = [doc.split() for doc in df['Title']]  

In [ ]:
import pickle
pickle.dump(doc_clean, open('dic.pickle', 'wb'))

In [ ]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
dictionary = corpora.Dictionary(doc_clean)
#dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [ ]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=1)

In [ ]:
for idx , topic in ldamodel.print_topics(-1):
    print(f"Topic : {idx} \n Words {topic} \n")

Topic : 0 
 Words 0.009*"variable" + 0.008*".net" + 0.007*"data" + 0.007*"table" + 0.007*"-" + 0.006*"php" + 0.006*"java" + 0.005*"menu" + 0.005*"last" + 0.005*"python" 

Topic : 1 
 Words 0.012*"way" + 0.010*"visual" + 0.010*"studio" + 0.010*"best" + 0.008*"jquery" + 0.008*"using" + 0.007*"file" + 0.007*"change" + 0.007*"source" + 0.007*"asp.net" 

Topic : 2 
 Words 0.026*"sql" + 0.022*"server" + 0.011*"asp.net" + 0.011*"get" + 0.010*"control" + 0.008*"using" + 0.008*"c#" + 0.008*"service" + 0.007*"2005" + 0.006*"code" 

Topic : 3 
 Words 0.015*"using" + 0.010*"&" + 0.009*"best" + 0.008*"user" + 0.007*"use" + 0.007*"sharepoint" + 0.005*"file" + 0.005*"php" + 0.005*"web" + 0.005*"issue" 

Topic : 4 
 Words 0.017*"using" + 0.015*"best" + 0.010*"web" + 0.010*"java" + 0.008*"way" + 0.007*"files" + 0.007*"database" + 0.006*"-" + 0.006*"array" + 0.005*"application" 

Topic : 5 
 Words 0.020*"use" + 0.014*"way" + 0.009*"-" + 0.009*"python" + 0.007*"best" + 0.005*"application" + 0.005*"access

In [ ]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=doc_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4880146204343411


Higher the coherence score , more separable the topics are, better it is . Trying with Multicore Model now -

In [ ]:
lda_tfidf = gensim.models.LdaMulticore(doc_term_matrix, num_topics = 10, id2word = dictionary, passes = 15, workers = 4, per_word_topics = True, chunksize = 3000)

In [ ]:
for idx, topic in lda_tfidf.print_topics(-1):
    print(f"Topic : {idx} \n Words {topic} \n")

Topic : 0 
 Words 0.009*"multiple" + 0.009*".net" + 0.006*"class" + 0.006*"c#" + 0.005*"name" + 0.005*"java" + 0.005*"using" + 0.005*"access" + 0.004*"database" + 0.004*"find" 

Topic : 1 
 Words 0.018*"way" + 0.018*"best" + 0.009*"-" + 0.008*"use" + 0.008*"what's" + 0.007*"java" + 0.007*"control" + 0.006*"store" + 0.006*"asp.net" + 0.005*"using" 

Topic : 2 
 Words 0.019*"server" + 0.013*"sql" + 0.013*"web" + 0.013*"asp.net" + 0.010*"file" + 0.006*"get" + 0.005*"project" + 0.005*"good" + 0.005*"unit" + 0.005*"create" 

Topic : 3 
 Words 0.023*"using" + 0.005*"php" + 0.005*"get" + 0.005*"java" + 0.005*"data" + 0.004*".net" + 0.004*"page" + 0.004*"vs." + 0.004*"vs" + 0.004*"call" 

Topic : 4 
 Words 0.008*"visual" + 0.007*"use" + 0.007*"get" + 0.006*"array" + 0.006*"type" + 0.005*"file" + 0.005*"linq" + 0.005*"way" + 0.005*"studio" + 0.005*"find" 

Topic : 5 
 Words 0.014*"best" + 0.011*"c#" + 0.010*"using" + 0.008*"web" + 0.007*"-" + 0.007*"method" + 0.007*"application" + 0.005*"php" +

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_tfidf, texts=doc_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4536393659621747
